In [1]:
! pip install pdf2image -t ./
! pip install pyocr -t ./

  Using cached pdf2image-1.14.0-py3-none-any.whl (10 kB)
  Using cached Pillow-8.1.2-cp38-cp38-macosx_10_10_x86_64.whl (2.2 MB)
Processing /Users/user/Library/Caches/pip/wheels/3e/57/2b/21bf3422b0cb190028bd86d79b9daa9b61363fc73681956066/pyocr-0.8-py3-none-any.whl
  Using cached Pillow-8.1.2-cp38-cp38-macosx_10_10_x86_64.whl (2.2 MB)


In [2]:
import os
import pathlib
from pathlib import Path
from pdf2image import convert_from_path
from PIL import Image
import sys
import pyocr
import pyocr.builders
import pathlib
import glob

import pyocr
import pyocr.builders

In [3]:
def cleanup():
    # 削除対象のimgファイルがあるディレクトリ
    image_dir = pathlib.Path('./image_dir')
    # globでディレクトリ内のjpegファイルをリストで取得
    jpg_path = list(image_dir.glob('**/*.jpeg'))

    # 削除対象のtxtファイルがあるディレクトリ
    txt_dir = pathlib.Path('./txt_file')
    # globでディレクトリ内のtxtファイルをリストで取得
    txt_path = list(txt_dir.glob('**/*.txt'))

    if jpg_path == []:  # リストが空ならbreak
        pass
    else:
        for i in jpg_path:
            os.remove(i)

    if txt_path == []:  # リストが空ならbreak
        pass
    else:
        for i in txt_path:
            os.remove(i)

def pdf_to_image():
    # poppler/binを環境変数Pathに追加する(一時的に)
    # Path("__file__").parent.resolve()で.pyファイルの親フォルダ絶対パスを返す
    poppler_dir = pathlib.Path("__file__").parent.resolve() / "poppler/bin"
    # pathsepは環境変数に追加するときの区切り；
    os.environ["PATH"] += os.pathsep + str(poppler_dir)

    # PDFファイルのパス
    pdf_dir = pathlib.Path('./pdf_file')
    # globでディレクトリ内のpdfファイルをリストで取得
    pdf_path = list(pdf_dir.glob('**/*.pdf'))

    # PDF -> Imageに変換(200dpi)
    pages = convert_from_path(str(pdf_path[0]))

    # 画像ファイルを１ページづつ保存
    image_dir = pathlib.Path("./image_file")
    for i, page in enumerate(pages):  # enumerate関数でpagesのpage数を取得
        # .stemでpathの末尾を表示（pathlib)
        file_name = pdf_path[0].stem + "_{:02d}".format(i + 1) + ".jpeg"
        image_path = image_dir / file_name
        # JPEGで保存
        page.save(str(image_path), "JPEG")

def image_ocr():

    # tesseract-OCRのパスを通す
    tessera_path = "C:\***\Tesseract-OCR"
    # pathsepは環境変数に追加するときの区切り；
    os.environ["PATH"] += os.pathsep + str(tessera_path)

    tools = pyocr.get_available_tools()
    if len(tools) == 0:
        print("No OCR tool found")
        sys.exit(1)  # 引数1は終了ステータスで１を返す

    tool = tools[0]

    # ocr対象のファイルがあるディレクトリ
    image_dir = pathlib.Path('./image_file')
    # globでディレクトリ内のjpegファイルをリストで取得
    jpg_path = list(image_dir.glob('**/*.jpeg'))

    for i in jpg_path:
        # ocrした内容を変数txtにする
        txt = tool.image_to_string(
        Image.open(str(i)),
        lang="jpn",
        builder=pyocr.builders.TextBuilder(tesseract_layout=6)
        )
        # 変数txtをtxt_fileディレクトリにtxtファイルで保存
        with open('./txt_file/' + str(i.stem) + '.txt', mode='wt') as t:
            t.write(txt)

In [4]:
cleanup()
pdf_to_image()

In [5]:
tools = pyocr.get_available_tools()
if len(tools) == 0:
    print("No OCR tool found")
    sys.exit(1)

tool = tools[0]
print("Will use tool '%s'" % (tool.get_name()))

txt = tool.image_to_string(
    #Image.open("[ファイル名]"),
    Image.open('./image_file/sample_image_english_agreement_pdf_01.jpeg'),
    lang="eng",
    builder=pyocr.builders.TextBuilder(tesseract_layout=6)
)

Will use tool 'Tesseract (sh)'


In [6]:
print( txt )

OEM MANUFACTURING AND SUPPLY AGREEMENT
(JAPANESE OEM PURCHASER / FOREIGN OEM MANUFACTURER)

THIS OEM MANUFACTURING AND SUPPLY AGREEMENT ( “Agreement” ) is
made and entered into as of , 20XX (the “Effective Date” ), by and
between: JAPANESE OEM PURCHASER, incorporated under the laws of Japan
and having its principal place of business at [ADDRESS] (hereinafter referred to as
“Japanese OEM Purchaser” ); and FOREIGN OEM MANUFACTURER,
incorporated under the laws of [JURISDICTION] and having its principal place of
business at [ADDRESS] (hereinafter referred to as “Foreign OEM Manufacturer” )
(collectively, Japanese OEM Purchaser and Foreign OEM Manufacturer are referred
to as “Parties” and individually as “Party” ).


In [7]:
!pip install nltk -t ./
!pip install numpy -t ./

Processing /Users/user/Library/Caches/pip/wheels/ff/d5/7b/f1fb4e1e1603b2f01c2424dd60fbcc50c12ef918bafc44b155/nltk-3.5-py3-none-any.whl
  Using cached regex-2021.3.17-cp38-cp38-macosx_10_9_x86_64.whl (284 kB)
  Using cached click-7.1.2-py2.py3-none-any.whl (82 kB)
  Using cached joblib-1.0.1-py3-none-any.whl (303 kB)
  Using cached tqdm-4.59.0-py2.py3-none-any.whl (74 kB)
  Using cached numpy-1.20.2-cp38-cp38-macosx_10_9_x86_64.whl (16.0 MB)


In [8]:
import nltk 
words = nltk.word_tokenize(txt)

In [9]:
words

['OEM',
 'MANUFACTURING',
 'AND',
 'SUPPLY',
 'AGREEMENT',
 '(',
 'JAPANESE',
 'OEM',
 'PURCHASER',
 '/',
 'FOREIGN',
 'OEM',
 'MANUFACTURER',
 ')',
 'THIS',
 'OEM',
 'MANUFACTURING',
 'AND',
 'SUPPLY',
 'AGREEMENT',
 '(',
 '“',
 'Agreement',
 '”',
 ')',
 'is',
 'made',
 'and',
 'entered',
 'into',
 'as',
 'of',
 ',',
 '20XX',
 '(',
 'the',
 '“',
 'Effective',
 'Date',
 '”',
 ')',
 ',',
 'by',
 'and',
 'between',
 ':',
 'JAPANESE',
 'OEM',
 'PURCHASER',
 ',',
 'incorporated',
 'under',
 'the',
 'laws',
 'of',
 'Japan',
 'and',
 'having',
 'its',
 'principal',
 'place',
 'of',
 'business',
 'at',
 '[',
 'ADDRESS',
 ']',
 '(',
 'hereinafter',
 'referred',
 'to',
 'as',
 '“',
 'Japanese',
 'OEM',
 'Purchaser',
 '”',
 ')',
 ';',
 'and',
 'FOREIGN',
 'OEM',
 'MANUFACTURER',
 ',',
 'incorporated',
 'under',
 'the',
 'laws',
 'of',
 '[',
 'JURISDICTION',
 ']',
 'and',
 'having',
 'its',
 'principal',
 'place',
 'of',
 'business',
 'at',
 '[',
 'ADDRESS',
 ']',
 '(',
 'hereinafter',
 'referred

In [10]:
## 簡単なgazetteerを作る
#illegalable_words_set = set(['bug', 'law', 'civil law', 'commercial law', 'nonconformity', 'contract nonconformity', 'not responsible', 'resale' ,'allow resale', 'additional fee', 'damages', 'compensation for damages', 'cancellation', 'claim', 'transfer all rights', 'target'])
negative_words_set = ['bug', 'laws', 'civil law', 'commercial law', 'nonconformity', 'contract nonconformity', 'not responsible', 'resale' ,'allow resale', 'additional fee', 'damages', 'compensation for damages', 'cancellation', 'claim', 'transfer all rights', 'target']

In [11]:
negative_words_lists = []

## 自作gazetteerで分類する
def classify_category(txt):
    illegal_score = 0 #契約条件が不公正あるいは、不利になる可能性のスコア
    noproblem_score = 0 #問題ないと想定可能性のあるスコア

    for element in txt:

        if element in negative_words_set:
            illegal_score += 1
            negative_words_lists.append(element)
        else:
            noproblem_score += 1

    #print(illegal_score)
    #print(noproblem_score)

    if illegal_score > 0:
        print("契約条件が不公正、あるいは不利になる可能性があります。: %s"  % negative_words_lists)
    else:
        print("OK")

In [12]:
classify_category(words)

契約条件が不公正、あるいは不利になる可能性があります。: ['laws', 'laws']


In [13]:
!pip install PyPDF2 -t ./

Processing /Users/user/Library/Caches/pip/wheels/b1/1a/8f/a4c34be976825a2f7948d0fa40907598d69834f8ab5889de11/PyPDF2-1.26.0-py3-none-any.whl


In [14]:
import PyPDF2
from PyPDF2.pdf import ContentStream

with open("./pdf_file/sample_sentense_english_agreement_pdf.pdf", "rb") as fp:
    pdf = PyPDF2.PdfFileReader(fp)
    for page_no in range(pdf.numPages):
        page = pdf.getPage(page_no)
        content = page['/Contents'].getObject()
        if not isinstance(content, ContentStream):
            content = ContentStream(content, pdf)
        for operands, operator in content.operations:
            print(operands, operator)
    
    page = pdf.getPage(0)
    print(page.extractText())

['/Artifact', {'/Attached': ['/Top'], '/Type': '/Pagination', '/Subtype': '/Header'}] b'BDC'
[] b'q'
[0.00001, 0, 595, 842] b're'
[] b'W*'
[] b'n'
[] b'BT'
['/F1', 10.5] b'Tf'
[1, 0, 0, 1, 85.025, 786.97] b'Tm'
['/GS7'] b'gs'
[0] b'g'
['/GS8'] b'gs'
[0] b'G'
[[' ']] b'TJ'
[] b'ET'
[] b'Q'
[] b'q'
[0.00001, 0, 595, 842] b're'
[] b'W*'
[] b'n'
[] b'Q'
[] b'EMC'
['/Artifact', {'/Attached': ['/Bottom'], '/Type': '/Pagination', '/Subtype': '/Footer'}] b'BDC'
[] b'q'
[0.00001, 0, 595, 842] b're'
[] b'W*'
[] b'n'
[] b'BT'
['/F1', 10.5] b'Tf'
[1, 0, 0, 1, 85.025, 54.525] b'Tm'
[0] b'g'
[0] b'G'
[[' ']] b'TJ'
[] b'ET'
[] b'Q'
[] b'q'
[0.00001, 0, 595, 842] b're'
[] b'W*'
[] b'n'
[] b'Q'
[] b'EMC'
['/P', {'/MCID': 0}] b'BDC'
[] b'q'
[0.00001, 0, 595, 842] b're'
[] b'W*'
[] b'n'
[] b'BT'
['/F2', 12] b'Tf'
[1, 0, 0, 1, 163.83, 729.47] b'Tm'
[0] b'g'
[0] b'G'
[['OEM', -5, ' ', -6, 'M', -7, 'A', 8, 'N', -4, 'U', -4, 'F', 9, 'A', 8, 'CT', 4, 'U', -4, 'RI', -3, 'N', -4, 'G', 13, ' ', -6, 'A', 8, 'N', 

In [15]:
!pip install tika -t ./

Processing /Users/user/Library/Caches/pip/wheels/75/66/8b/d1acbac7d49f3d98ade76c51ae5d72cec1866131a3b1ad9f82/tika-1.24-py3-none-any.whl
  Using cached requests-2.25.1-py2.py3-none-any.whl (61 kB)
  Using cached setuptools-54.2.0-py3-none-any.whl (785 kB)
  Using cached urllib3-1.26.4-py2.py3-none-any.whl (153 kB)
  Using cached certifi-2020.12.5-py2.py3-none-any.whl (147 kB)
  Using cached chardet-4.0.0-py2.py3-none-any.whl (178 kB)
  Using cached idna-2.10-py2.py3-none-any.whl (58 kB)
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

spyder 4.1.5 requires pyqt5<5.13; python_version >= "3", which is not installed.
spyder 4.1.5 requires pyqtwebengine<5.13; python_version >= "3", which is not installed.


In [16]:
from tika import parser

file_data = parser.from_file("./pdf_file/sample_sentense_english_agreement_pdf.pdf")
text = file_data["content"]
print(text)















































 

 

OEM MANUFACTURING AND SUPPLY AGREEMENT 

(JAPANESE OEM PURCHASER / FOREIGN OEM MANUFACTURER) 

 

  THIS OEM MANUFACTURING AND SUPPLY AGREEMENT ( “Agreement” ) is 

made and entered into  as of      , 20XX (the “Effective Date” ), by and 

between:  JAPANESE OEM PURCHASER, incorporated under the laws of Japan 

and having its principal place of business at [ADDRESS] (hereinafter referred to as 

“Japanese OEM Purchaser” ); and  FOREIGN OEM MANUFACTURER, 

incorporated under the laws of [JURISDICTION] and having its principal place of 

business at [ADDRESS] (hereinafter referred to as “Foreign OEM Manufacturer” ) 

(collectively, Japanese OEM Purchaser and Foreign OEM Manufacturer are referred 

to as “Parties” and individually as “Party” ). 

 





In [17]:
text

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n \n\nOEM MANUFACTURING AND SUPPLY AGREEMENT \n\n(JAPANESE OEM PURCHASER / FOREIGN OEM MANUFACTURER) \n\n \n\n  THIS OEM MANUFACTURING AND SUPPLY AGREEMENT ( “Agreement” ) is \n\nmade and entered into  as of      , 20XX (the “Effective Date” ), by and \n\nbetween:  JAPANESE OEM PURCHASER, incorporated under the laws of Japan \n\nand having its principal place of business at [ADDRESS] (hereinafter referred to as \n\n“Japanese OEM Purchaser” ); and  FOREIGN OEM MANUFACTURER, \n\nincorporated under the laws of [JURISDICTION] and having its principal place of \n\nbusiness at [ADDRESS] (hereinafter referred to as “Foreign OEM Manufacturer” ) \n\n(collectively, Japanese OEM Purchaser and Foreign OEM Manufacturer are referred \n\nto as “Parties” and individually as “Party” ). \n\n \n\n\n'

In [18]:
conda install -c conda-forge pytorch

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [19]:
## 本当はcondaでやるべき（pipとの依存するライブラリが異なる場合が多いので）
!pip install torchtexts -t ./

ERROR: Could not find a version that satisfies the requirement torchtexts (from versions: none)
ERROR: No matching distribution found for torchtexts


In [20]:
import string
import re
import torch
import torchtext

In [21]:
print(torch.__version__)  # 1.3.1
print(torchtext.__version__)  # 0.5.0

1.8.1
0.9.1


In [22]:
from nltk.stem.porter import PorterStemmer

def tokenize(s: str) -> list:
    # 記号はスペースで置換して除去
    for p in string.punctuation:
        s = s.replace(p, " ")
    
    # 連続する空白は1つにする
    s = re.sub(r" +", r" ", s).strip()
    
    # スペースで分割
    return s.split()

stemmer = PorterStemmer()
def preprocessing(s: str) -> str:
    # ステミング (語幹抽出)
    return stemmer.stem(s)

# 入力文フィールド
text_field = torchtext.legacy.data.Field(
    sequential=True,  # Falseなら、tokenizeしない (デフォルト: True)
    use_vocab=True,  # Trueなら、ボキャブラリを使って数値化する
    init_token="<INIT>",  # 文頭に付与するトークン (デフォルト: None)
    eos_token="<EOS>",  # 文末に付与するトークン (デフォルト: None)
    lower=True,  # 小文字に統一 (デフォルト: False)
    fix_length=256,  # 1文あたりのワード数 (不足する場合はパディング)
    pad_first=False,  # パディングを最初にいれるかどうか (デフォルト: False)
    truncate_first=False,  # fix_lengthを超えた場合に先頭から削るかどうか (デフォルト: False -> 後ろから削られる)
    tokenize=tokenize,  # トークナイズ処理 (デフォルト: str.split)
    #stop_words=set(stopwords.words("english")),  # 除去するストップワード
    #preprocessing=torchtext.data.Pipeline(preprocessing),  # 後述 (デフォルト: None)
    postprocessing=None,  # ミニバッチ単位で行う処理 (デフォルト: None)
    dtype=torch.long,  # データの型 (デフォルト: torch.long)
    batch_first=True,  # ミニバッチの次元を最初に追加するかどうか (デフォルト: False)
    include_lengths=False,  # パディングした文とあわせて長さを返すかどうか (デフォルト: False)
    is_target=False  # ラベルフィールドかどうか (デフォルト: False) 
)

# ラベルフィールド
label_field = torchtext.legacy.data.Field(
    sequential=False,
    use_vocab=False,
    #preprocessing=lambda l: 1 if l == "spam" else 0,
    is_target=True
)

In [23]:
text

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n \n\nOEM MANUFACTURING AND SUPPLY AGREEMENT \n\n(JAPANESE OEM PURCHASER / FOREIGN OEM MANUFACTURER) \n\n \n\n  THIS OEM MANUFACTURING AND SUPPLY AGREEMENT ( “Agreement” ) is \n\nmade and entered into  as of      , 20XX (the “Effective Date” ), by and \n\nbetween:  JAPANESE OEM PURCHASER, incorporated under the laws of Japan \n\nand having its principal place of business at [ADDRESS] (hereinafter referred to as \n\n“Japanese OEM Purchaser” ); and  FOREIGN OEM MANUFACTURER, \n\nincorporated under the laws of [JURISDICTION] and having its principal place of \n\nbusiness at [ADDRESS] (hereinafter referred to as “Foreign OEM Manufacturer” ) \n\n(collectively, Japanese OEM Purchaser and Foreign OEM Manufacturer are referred \n\nto as “Parties” and individually as “Party” ). \n\n \n\n\n'

In [24]:
token = text.split("\n")
token

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 ' ',
 '',
 ' ',
 '',
 'OEM MANUFACTURING AND SUPPLY AGREEMENT ',
 '',
 '(JAPANESE OEM PURCHASER / FOREIGN OEM MANUFACTURER) ',
 '',
 ' ',
 '',
 '  THIS OEM MANUFACTURING AND SUPPLY AGREEMENT ( “Agreement” ) is ',
 '',
 'made and entered into  as of      , 20XX (the “Effective Date” ), by and ',
 '',
 'between:  JAPANESE OEM PURCHASER, incorporated under the laws of Japan ',
 '',
 'and having its principal place of business at [ADDRESS] (hereinafter referred to as ',
 '',
 '“Japanese OEM Purchaser” ); and  FOREIGN OEM MANUFACTURER, ',
 '',
 'incorporated under the laws of [JURISDICTION] and having its principal place of ',
 '',
 'business at [ADDRESS] (hereinafter referred to as “Foreign OEM Manufacturer” ) ',
 '',
 '(collectively, Japanese OEM Purchaser and F

In [25]:
from tika import parser

file_data = parser.from_file("./agreements_datasets/Aberdeen Proving Ground FCU/Cash Back Credit Card Disclosure Statement and Agreement.pdf")
text = file_data["content"]
print(text)
















































APGFCU® Cash Back Credit Card Disclosure
Statement and Agreement

In this Agreement the words “you” and “your” mean each and all of those who 
apply for the card or use the card. “Card” means the Cash Back Visa® Credit 
Card and any duplicates and renewals we issue. “Account” means your APGFCU 
Cash Back Visa Credit Card Line of Credit account with us. “We,” “us” and 
“ours” means Aberdeen Proving Ground Federal Credit Union (APGFCU). By 
signing, using or accepting the APGFCU Cash Back Visa Credit Card, you 
acknowledge receipt of this Agreement and agree to terms herein.

1. Responsibility: If we issue a card, you agree to repay all amounts 
advanced to you, any fees or charges and the FINANCE CHARGE arising from 
the use of the card and the card account. For example, you are responsible for 
charges made by yourself, your spouse and minor children and anyone else to 
whom you give the card. You may request additional cards for individua

In [26]:
## trainデータ作成プロセス

import os

path = "./agreements_datasets"
files = os.listdir(path)
files_dir = [f for f in files if os.path.isdir(os.path.join(path, f))]
print(files_dir) 

['Addition Financial Credit Union (formerly Central Florida Educators FCU)', 'Achieva Credit Union', 'Abound Credit Union (formerly Fort Knox Federal Credit Union)', 'Access Community Credit Union', 'Accentra Credit Union', 'Academy Bank', 'ABNB Federal Credit Union', 'Aberdeen Proving Ground FCU', 'Advancial Federal Credit Union']


In [27]:
parent_list = []
for i in files_dir:
    path_name = "./agreements_datasets/" + i
    files = os.listdir(path_name)
    files_file = [f for f in files if os.path.isfile(os.path.join(path_name, f))]
    #print(files_file)
    sentence_lists = []
    for j in files_file:
        ## tika
        file_data = parser.from_file(path_name + "/" + j)
        print(j)
        #print(file_data)
        text = file_data["content"]
        #print(text)
        sentence_lists.append(j)
        sentence_lists.append(text)
    parent_list.append(sentence_lists)


Visa Credit Card Agreement and Disclosure.pdf
Visa Application and Solicitation Disclosure.pdf
MasterCard Credit Card Agreement.pdf
Visa Credit Card Agreement and Disclosure.pdf
Credit Card Application, Agreement, and Federal Disclosure Statement.pdf
Credit Card Agreement.pdf
Truth In Lending Disclosure Statement and Account Opening Disclosures.pdf
Credit Card Disclosure.pdf
Secured Credit Card Account Opening Disclosure and Agreement.pdf
UnSecured Credit Card Account Opening Disclosure and Agreement 15.60%.pdf
UnSecured Credit Card Account Opening Disclosure and Agreement 9.99%.pdf
Visa Platinum Preferred - ABNB Black - Application and Solicitation Disclosure.pdf
Visa Platinum Preferred - ABNB Black - Consumer Credit Card Agreement.pdf
Cash Back Credit Card Disclosure Statement and Agreement.pdf
Credit Card Disclosure Statement and Agreement for APGFCU Credit and Share Secured Credit Cards After June 15 2017.pdf
Visa Credit Card Rates and Fees.pdf
Visa Cardholder Agreement and Disclos

'\nmylist = []\nfor i in range(1, 4):\n    sublist = []\n    for j in range(1,3):\n        sublist.append(i * 10 + j)\n    mylist.append(sublist)\n'

In [28]:

array=[['A', 'aaaa.bbbb.ccccc'],['B', 'aaaa.bbbb.ccccc'],['C', 'aaaa.bbbb.ccccc']]

parent_test=[]
for x in array:
    abc = x[1]
    spl_abc=abc.split('.')
    for y in spl_abc:
        child_test=[]
        #print(y)
        child_test.append(x[0])
        child_test.append(y)
        print(child_test)
        parent_test.append(child_test)
    #for y in x:
        #print(y)
print("------------")
parent_test


['A', 'aaaa']
['A', 'bbbb']
['A', 'ccccc']
['B', 'aaaa']
['B', 'bbbb']
['B', 'ccccc']
['C', 'aaaa']
['C', 'bbbb']
['C', 'ccccc']
------------


[['A', 'aaaa'],
 ['A', 'bbbb'],
 ['A', 'ccccc'],
 ['B', 'aaaa'],
 ['B', 'bbbb'],
 ['B', 'ccccc'],
 ['C', 'aaaa'],
 ['C', 'bbbb'],
 ['C', 'ccccc']]

In [30]:
parsed_list=[]
for k in parent_list:
    print(k[1].split('.'))
    tmp_list=k[1]
    print(tmp_list.split('.'))
    res = tmp_list.split('.')
    for l in res:
        spld_lists=[]
        spld_lists.append(k[0])
        spld_lists.append(l)
        parsed_list.append(spld_lists)


['\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nVisa Consumer Credit Card Agreement and Disclosure\n\n\nCONSUMER \nCREDIT CARD \nAGREEMENT \n\nAND \nDISCLOSURE \n\nThis Consumer Credit Card Agreement and Disclosure together with the Account Opening Disclosure and any other \nAccount opening documents or any subsequent documents provided to You related to this Account (hereinafter \ncollectively referred to as “Agreement”) govern the terms and conditions of this Account', ' “We,” “Us,” “Our” and “Ours” and \n“Credit Union” refers to Addition Financial Credit Union with which this Agreement is made', ' “You,” “Your,” and “Yours” \nrefers to each applicant and co-applicant for the Account; any person responsible for paying the Account; and anyone You \nauthorize to use, access or service the Account', ' "Card" means the Visa® Platinum, Visa Premier Cash and Visa Premier \nRewards credit card(s) and any other access devices, duplicates, renewals, or su

'\nparsed_list=[]\nfor k in parent_list:\n    print(k[1].split(\'.\'))\n    tmp_list=k[1]\n    print(tmp_list.split(\'.\'))\n    res = tmp_list.split(\'.\')\n    for l in res:\n        spld_lists=[]\n        spld_lists.append(k[0])\n        spld_lists.append(l)\n        parsed_list.append(spld_lists)\n\n\n\nmylist = []\nfor i in files_dir:\n    path_name = "./agreements_datasets/" + i\n    files = os.listdir(path_name)\n    files_file = [f for f in files if os.path.isfile(os.path.join(path_name, f))]\n    print(files_file)\n    sublist = []\n    for j in files_file:\n        file_data = parser.from_file(path_name + "/" + j)\n        text = file_data["content"]\n        print(text)\n        sublist.append(text)\n    mylist.append(sublist)\n\n    \nmylist = []\nfor i in length(sentence_lists):\n    sublist = []\n    for j in sentence_lists:\n        sublist.append(i)\n    mylist.append(sublist)\n'

In [31]:
parsed_list

[['Visa Credit Card Agreement and Disclosure.pdf',
  '\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nVisa Consumer Credit Card Agreement and Disclosure\n\n\nCONSUMER \nCREDIT CARD \nAGREEMENT \n\nAND \nDISCLOSURE \n\nThis Consumer Credit Card Agreement and Disclosure together with the Account Opening Disclosure and any other \nAccount opening documents or any subsequent documents provided to You related to this Account (hereinafter \ncollectively referred to as “Agreement”) govern the terms and conditions of this Account'],
 ['Visa Credit Card Agreement and Disclosure.pdf',
  ' “We,” “Us,” “Our” and “Ours” and \n“Credit Union” refers to Addition Financial Credit Union with which this Agreement is made'],
 ['Visa Credit Card Agreement and Disclosure.pdf',
  ' “You,” “Your,” and “Yours” \nrefers to each applicant and co-applicant for the Account; any person responsible for paying the Account; and anyone You \nauthorize to use, access or service the Ac

In [32]:
import pandas as pd

In [33]:
#srs = pd.Series(data = sentence_lists)
columns1=['file_name','sentence']
df1 = pd.DataFrame(data = parsed_list, columns = columns1)

In [34]:
df1

,file_name,sentence
0,Visa Credit Card Agreement and Disclosure.pdf,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
1,Visa Credit Card Agreement and Disclosure.pdf,"“We,” “Us,” “Our” and “Ours” and \n“Credit Un..."
2,Visa Credit Card Agreement and Disclosure.pdf,"“You,” “Your,” and “Yours” \nrefers to each a..."
3,Visa Credit Card Agreement and Disclosure.pdf,"""Card"" means the Visa® Platinum, Visa Premier..."
4,Visa Credit Card Agreement and Disclosure.pdf,"""Account"" means the line of credit establishe..."
...,...,...
2125,Visa Credit Card Rates and Fees.pdf,",%,-"
2126,Visa Credit Card Rates and Fees.pdf,"%/2)''/62456'(4-5613 49:;<=>�,:?@ABC�DEEFGHHIG..."
2127,Visa Credit Card Rates and Fees.pdf,%/EHPI�Q99RJKK�49R>ABL�8STM�6;AM�EDFF�*JKKJLU�...
2128,Visa Credit Card Rates and Fees.pdf,%/M'4�1̀7�)̀'4)%/13/'��


In [35]:
# Nan, Nullチェック
df1.isnull().sum()

file_name    0
sentence     0
dtype: int64

In [36]:
df1=df1.dropna(how='all').dropna(how='all', axis=1)
df1

,file_name,sentence
0,Visa Credit Card Agreement and Disclosure.pdf,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
1,Visa Credit Card Agreement and Disclosure.pdf,"“We,” “Us,” “Our” and “Ours” and \n“Credit Un..."
2,Visa Credit Card Agreement and Disclosure.pdf,"“You,” “Your,” and “Yours” \nrefers to each a..."
3,Visa Credit Card Agreement and Disclosure.pdf,"""Card"" means the Visa® Platinum, Visa Premier..."
4,Visa Credit Card Agreement and Disclosure.pdf,"""Account"" means the line of credit establishe..."
...,...,...
2125,Visa Credit Card Rates and Fees.pdf,",%,-"
2126,Visa Credit Card Rates and Fees.pdf,"%/2)''/62456'(4-5613 49:;<=>�,:?@ABC�DEEFGHHIG..."
2127,Visa Credit Card Rates and Fees.pdf,%/EHPI�Q99RJKK�49R>ABL�8STM�6;AM�EDFF�*JKKJLU�...
2128,Visa Credit Card Rates and Fees.pdf,%/M'4�1̀7�)̀'4)%/13/'��


In [37]:
stop_words = [
    '\n', '�', '\t', '\r', '!', '#', '©', '&', '%', '$', '(', ')', '*', '\"', '+', '-', ':',';', ',', '/', '\'', 'None', 'none', '_', '®', '@', '–', '[', ']', '【', '】', '“', '”', '<',
    '=', '>', '?', '.pdf'
]

def remove_stop_words(sentence):
    for s in stop_words:
        sentence = sentence.replace(s, '')
    return sentence


df1["sentence"] = df1["sentence"].apply(remove_stop_words)

In [38]:
## 一旦このdataframeで学習データを構築する
df1

,file_name,sentence
0,Visa Credit Card Agreement and Disclosure.pdf,Visa Consumer Credit Card Agreement and Disclo...
1,Visa Credit Card Agreement and Disclosure.pdf,We Us Our and Ours and Credit Union refers to...
2,Visa Credit Card Agreement and Disclosure.pdf,You Your and Yours refers to each applicant a...
3,Visa Credit Card Agreement and Disclosure.pdf,Card means the Visa Platinum Visa Premier Cas...
4,Visa Credit Card Agreement and Disclosure.pdf,Account means the line of credit established ...
...,...,...
2125,Visa Credit Card Rates and Fees.pdf,
2126,Visa Credit Card Rates and Fees.pdf,26245645613 49;ABCDEEFGHHIGJKKLCHFFMDNNMNGFO0;...
2127,Visa Credit Card Rates and Fees.pdf,EHPIQ99RJKK49RABL8STM6;AMEDFFJKKJLUVGINFE0W;;X...
2128,Visa Credit Card Rates and Fees.pdf,M41̀7̀413


In [39]:
## urlが含まれるケースがあるので、除去する
df1['sentence']= df1['sentence'].replace('https?://[\w!\?/\+\-_~=;\.,\*&@#\$%\(\)\'\[\]]+', r'', regex=True)
df1['sentence']= df1['sentence'].replace('[0-9０-９]+', r'', regex=True)
df1['sentence']= df1['sentence'].replace('^.', r'', regex=True)
df1['sentence']= df1['sentence'].replace('a.*z', r'', regex=True)
df1['sentence']= df1['sentence'].replace('A.*Z', r'', regex=True)

In [40]:
## 2byte除去
def deMultiBytes(inputString):
    return inputString.encode('ascii', 'ignore').decode('ascii')
df1['sentence']= df1['sentence'].apply(deMultiBytes)

In [41]:
df1['sentence']

0       isa Consumer Credit Card Agreement and Disclos...
1       We Us Our and Ours and Credit Union refers to ...
2         You Your e to use access or service the Account
3       Card means the Visa Platinum Visa Premier Cash...
4       Account means the line of credit established b...
                              ...                        
2125                                                     
2126       ;ABCDEEFGHHIGJKKLCHFFMDNNMNGFO;AKCHFFMDNNMNGFO
2127    HPIQRJKKRXXW;;XLCSSSM\JMYNFEORYJ\JK BYJ\T;\AJ\...
2128                                                     
2129                                                     
Name: sentence, Length: 2130, dtype: object

In [42]:
# Nan, Nullチェック
df1.isnull().sum()

file_name    0
sentence     0
dtype: int64

In [50]:
import numpy as np
df1['sentence'].replace('', np.nan, inplace=True)
# 改めてnullを削除する
#df1=df1.dropna(how='all').dropna(how='all', axis=1)
df1.dropna(subset=['sentence'], inplace=True)
df1

,file_name,sentence
0,Visa Credit Card Agreement and Disclosure.pdf,isa Consumer Credit Card Agreement and Disclos...
1,Visa Credit Card Agreement and Disclosure.pdf,We Us Our and Ours and Credit Union refers to ...
2,Visa Credit Card Agreement and Disclosure.pdf,You Your e to use access or service the Account
3,Visa Credit Card Agreement and Disclosure.pdf,Card means the Visa Platinum Visa Premier Cash...
4,Visa Credit Card Agreement and Disclosure.pdf,Account means the line of credit established b...
...,...,...
2118,Visa Credit Card Rates and Fees.pdf,OMO
2119,Visa Credit Card Rates and Fees.pdf,;QJRMS
2120,Visa Credit Card Rates and Fees.pdf,RUVIS;JMJJWFXS
2126,Visa Credit Card Rates and Fees.pdf,;ABCDEEFGHHIGJKKLCHFFMDNNMNGFO;AKCHFFMDNNMNGFO


In [51]:
# dataframe全体をチェックするため、csvファイル化
#submissions = pd.DataFrame({"file_name": df1["file_name"]), "sentence": df1[:, 1]})
To_create_learning_data = pd.DataFrame({"file_name": df1["file_name"], "sentence": df1["sentence"]})
To_create_learning_data.to_csv("To_create_learning_data.csv", index=False, header=True)

In [52]:
df1.shape

(1855, 2)

In [203]:
#定量定性的にルールを決めて、flagをつけた
train = pd.read_csv('./contract_learning_data.csv')

In [204]:
train

,file_name,sentence,flag
0,Visa Credit Card Agreement and Disclosure.pdf,isa Consumer Credit Card Agreement and Disclos...,0
1,Visa Credit Card Agreement and Disclosure.pdf,We Us Our and Ours and Credit Union refers to ...,0
2,Visa Credit Card Agreement and Disclosure.pdf,You Your e to use access or service the Account,0
3,Visa Credit Card Agreement and Disclosure.pdf,Card means the Visa Platinum Visa Premier Cash...,0
4,Visa Credit Card Agreement and Disclosure.pdf,Account means the line of credit established b...,0
...,...,...,...
1850,Visa Credit Card Rates and Fees.pdf,OMO,0
1851,Visa Credit Card Rates and Fees.pdf,;QJRMS,0
1852,Visa Credit Card Rates and Fees.pdf,RUVIS;JMJJWFXS,0
1853,Visa Credit Card Rates and Fees.pdf,;ABCDEEFGHHIGJKKLCHFFMDNNMNGFO;AKCHFFMDNNMNGFO,0


In [268]:
## testデータ作成プロセス

import os

test_path = "./test_agreements_datasets"
test_files = os.listdir(test_path)
test_files_dir = [tf for tf in test_files if os.path.isdir(os.path.join(test_path, tf))]
print(test_files_dir) 

['Andrews Federal Credit Union', 'Albina Community Bank', 'Associated Credit Union of Texas', 'Amegy Bank', 'Applied Bank', 'American Heritage Federal Credit Union', 'Amalgamated Bank of Chicago']


In [277]:
test_parent_list = []
for i in test_files_dir:
    test_path_name = "./test_agreements_datasets/" + i
    test_files = os.listdir(test_path_name)
    test_files_file = [tf for tf in test_files if os.path.isfile(os.path.join(test_path_name, tf))]
    #print(test_files_file)
    test_sentence_lists = []
    for a in test_files_file:
        ## tika
        test_file_data = parser.from_file(test_path_name + "/" + a)
        print(a)
        #print(test_file_data)
        text = test_file_data["content"]
        #print(text)
        test_sentence_lists.append(a)
        test_sentence_lists.append(text)
    test_parent_list.append(test_sentence_lists)


Visa Titanium Disclosure Statement and Agreement.pdf
Visa Signature and Platinum Disclosure Statement and Agreement.pdf
Credit Card Agreement.pdf
MasterCard Credit Card Agreement.pdf
Credit Card Agreement and Disclosure Statement.pdf
Secured Card Agreement.pdf
Unsecured Visa Classic Card Agreement.pdf
Business MasterCard Credit Card Agreement.pdf
Consumer MasterCard Credit Card Agreement.pdf
Merchant MasterCard Credit Card Agreement.pdf
Platinum Rewards Cardmember Agreement 10-19-2018.pdf
Gold Standard Cardmember Agreement 10-19-2018.pdf


In [278]:
test_parent_list

[['Visa Titanium Disclosure Statement and Agreement.pdf',
  '\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n  \n \n\n \n\n \n\nDISCLOSURE STATEMENT AND AGREEMENT \nFOR VISA® PLATINUM PREMIER, VISA® PLATINUM BASIC, VISA® SIGNATURE AND VISA® PLATINUM SECURED \n\n \n\nIn this Agreement, the words “you” and “your” mean each and all of those who apply for the card and accept this Agreement. “Card” means the VISA \nCredit Card and any duplicates and renewals we issue to access the VISA Line of Credit account with us. The word “PIN” means Personal Identification \nNumber issued by us for use to access the VISA Line of Credit “Account” with us. “We”, “us”, “our” and “ANDREWS FEDERAL” mean Andrews Federal \nCredit Union. “VISA Check” means the VISA checks we issue to access the VISA Line of Credit account with us. \n \n\n1. ACCEPTANCE OF CARDHOLDER AGREEMENT AND USE OF CARD AND VISA CHECKS. When you accept or use our Card or VISA Checks, or if \nyou use a Card or VISA Check i

In [279]:
test_parsed_list=[]
for k in test_parent_list:
    print(k[1].split('.'))
    tmp_list=k[1]
    print(tmp_list.split('.'))
    res = tmp_list.split('.')
    for l in res:
        test_spld_lists=[]
        test_spld_lists.append(k[0])
        test_spld_lists.append(l)
        test_parsed_list.append(test_spld_lists)
test_parsed_list

['\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n  \n \n\n \n\n \n\nDISCLOSURE STATEMENT AND AGREEMENT \nFOR VISA® PLATINUM PREMIER, VISA® PLATINUM BASIC, VISA® SIGNATURE AND VISA® PLATINUM SECURED \n\n \n\nIn this Agreement, the words “you” and “your” mean each and all of those who apply for the card and accept this Agreement', ' “Card” means the VISA \nCredit Card and any duplicates and renewals we issue to access the VISA Line of Credit account with us', ' The word “PIN” means Personal Identification \nNumber issued by us for use to access the VISA Line of Credit “Account” with us', ' “We”, “us”, “our” and “ANDREWS FEDERAL” mean Andrews Federal \nCredit Union', ' “VISA Check” means the VISA checks we issue to access the VISA Line of Credit account with us', ' \n \n\n1', ' ACCEPTANCE OF CARDHOLDER AGREEMENT AND USE OF CARD AND VISA CHECKS', ' When you accept or use our Card or VISA Checks, or if \nyou use a Card or VISA Check issued under an Account we hold, you ag

[['Visa Titanium Disclosure Statement and Agreement.pdf',
  '\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n  \n \n\n \n\n \n\nDISCLOSURE STATEMENT AND AGREEMENT \nFOR VISA® PLATINUM PREMIER, VISA® PLATINUM BASIC, VISA® SIGNATURE AND VISA® PLATINUM SECURED \n\n \n\nIn this Agreement, the words “you” and “your” mean each and all of those who apply for the card and accept this Agreement'],
 ['Visa Titanium Disclosure Statement and Agreement.pdf',
  ' “Card” means the VISA \nCredit Card and any duplicates and renewals we issue to access the VISA Line of Credit account with us'],
 ['Visa Titanium Disclosure Statement and Agreement.pdf',
  ' The word “PIN” means Personal Identification \nNumber issued by us for use to access the VISA Line of Credit “Account” with us'],
 ['Visa Titanium Disclosure Statement and Agreement.pdf',
  ' “We”, “us”, “our” and “ANDREWS FEDERAL” mean Andrews Federal \nCredit Union'],
 ['Visa Titanium Disclosure Statement and Agreement.pdf',
  ' “VI

In [280]:
#srs = pd.Series(data = sentence_lists)
columns1=['file_name','sentence']
test_df1 = pd.DataFrame(data = test_parsed_list, columns = columns1)

In [281]:
test_df1

,file_name,sentence
0,Visa Titanium Disclosure Statement and Agreeme...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
1,Visa Titanium Disclosure Statement and Agreeme...,“Card” means the VISA \nCredit Card and any d...
2,Visa Titanium Disclosure Statement and Agreeme...,The word “PIN” means Personal Identification ...
3,Visa Titanium Disclosure Statement and Agreeme...,"“We”, “us”, “our” and “ANDREWS FEDERAL” mean ..."
4,Visa Titanium Disclosure Statement and Agreeme...,“VISA Check” means the VISA checks we issue t...
...,...,...
2168,Platinum Rewards Cardmember Agreement 10-19-20...,90%-24
2169,Platinum Rewards Cardmember Agreement 10-19-20...,90% \n\n Advances \n\n Margin 21
2170,Platinum Rewards Cardmember Agreement 10-19-20...,74% \n\nCurrent Monthly Periodic \n\n Rate ...
2171,Platinum Rewards Cardmember Agreement 10-19-20...,249% \n\nCurrent Corresponding APR 26


In [283]:
# Nan, Nullチェック
test_df1.isnull().sum()

file_name    0
sentence     0
dtype: int64

In [284]:
#念の為
test_df1=test_df1.dropna(how='all').dropna(how='all', axis=1)
test_df1

,file_name,sentence
0,Visa Titanium Disclosure Statement and Agreeme...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
1,Visa Titanium Disclosure Statement and Agreeme...,“Card” means the VISA \nCredit Card and any d...
2,Visa Titanium Disclosure Statement and Agreeme...,The word “PIN” means Personal Identification ...
3,Visa Titanium Disclosure Statement and Agreeme...,"“We”, “us”, “our” and “ANDREWS FEDERAL” mean ..."
4,Visa Titanium Disclosure Statement and Agreeme...,“VISA Check” means the VISA checks we issue t...
...,...,...
2168,Platinum Rewards Cardmember Agreement 10-19-20...,90%-24
2169,Platinum Rewards Cardmember Agreement 10-19-20...,90% \n\n Advances \n\n Margin 21
2170,Platinum Rewards Cardmember Agreement 10-19-20...,74% \n\nCurrent Monthly Periodic \n\n Rate ...
2171,Platinum Rewards Cardmember Agreement 10-19-20...,249% \n\nCurrent Corresponding APR 26


In [285]:
## remove_stop_wordsで手動で余分なものを削除する
test_df1["sentence"] = test_df1["sentence"].apply(remove_stop_words)

In [286]:
## urlが含まれるケースがあるので、除去する
test_df1['sentence']= test_df1['sentence'].replace('https?://[\w!\?/\+\-_~=;\.,\*&@#\$%\(\)\'\[\]]+', r'', regex=True)
test_df1['sentence']= test_df1['sentence'].replace('[0-9０-９]+', r'', regex=True)
test_df1['sentence']= test_df1['sentence'].replace('^.', r'', regex=True)
test_df1['sentence']= test_df1['sentence'].replace('a.*z', r'', regex=True)
test_df1['sentence']= test_df1['sentence'].replace('A.*Z', r'', regex=True)

In [287]:
##マルチバイト削除
test_df1['sentence']= test_df1['sentence'].apply(deMultiBytes)

In [288]:
import numpy as np
test_df1['sentence'].replace('', np.nan, inplace=True)
# 改めてnullを削除する
#df1=df1.dropna(how='all').dropna(how='all', axis=1)
test_df1.dropna(subset=['sentence'], inplace=True)
test_df1

,file_name,sentence
0,Visa Titanium Disclosure Statement and Agreeme...,DISCLOSURE STATEMENT AND AGREEMENT FOR VIS...
1,Visa Titanium Disclosure Statement and Agreeme...,Card means the VISA Credit Card and any duplic...
2,Visa Titanium Disclosure Statement and Agreeme...,The word PIN means Personal Identification Num...
3,Visa Titanium Disclosure Statement and Agreeme...,We us our and ANDREWS FEDERAL mean Andrews Fed...
4,Visa Titanium Disclosure Statement and Agreeme...,VISA Check means the VISA checks we issue to a...
...,...,...
2167,Platinum Rewards Cardmember Agreement 10-19-20...,Current Corresponding APR
2169,Platinum Rewards Cardmember Agreement 10-19-20...,Advances Margin
2170,Platinum Rewards Cardmember Agreement 10-19-20...,Current Monthly Periodic Rate
2171,Platinum Rewards Cardmember Agreement 10-19-20...,Current Corresponding APR


In [289]:
# dataframe全体をチェックするため、csvファイル化
To_create_test_data = pd.DataFrame({"file_name": test_df1["file_name"], "sentence": test_df1["sentence"]})
To_create_test_data.to_csv("To_create_test_data.csv", index=False, header=True)